In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torchvision.transforms as transforms
from Anchor.anchor import Anchor, Tasktype
from torchvision.models import inception_v3, resnet18
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import numpy as np
from skimage.segmentation import mark_boundaries

%load_ext autoreload
%autoreload 2

In [ ]:

# model:c, h, w
# numpy, sklearn matplotlib: w, h, c
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# c, h, w

image = Image.open("../static/dog_paper.jpeg")
input = preprocess(image).unsqueeze(0)
# b, c, h, w

model = inception_v3(pretrained=True)
model.eval()

plt.imshow(input.squeeze().permute(1,2,0))
# h w c

with torch.no_grad():
    output = model(input)

print(torch.argmax(output, dim=1))
probs = torch.nn.functional.softmax(output[0], dim=0)

with open("../static/imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
    # Show top categories per image
    top5_prob, top5_catid = torch.topk(probs, 5)
    top5_prob, top5_catid = top5_prob.squeeze(), top5_catid.squeeze()
    for i in range(top5_prob.size(0)):
        print(top5_catid[i])
        print(categories[top5_catid[i]], top5_prob[i].item())

In [ ]:
from Anchor.util import pytorch_image_wrapper

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

explainer = Anchor(Tasktype.IMAGE)

@pytorch_image_wrapper(device)
def predict(x):
    return model(x)


anchor = explainer.explain_instance(input.squeeze().permute(1,2,0), predict, "beam", None, num_coverage_samples=1000)

# img, sp_img = explainer.visualize()
# # Greedy segment test
# img2 = img.copy()
# mask = np.zeros(segments.shape).astype(bool)
# for z in (11, 19):
#     mask[segments == z] = True
# img2[~mask] = sp_img[~mask]

# fig = plt.figure(figsize=(8, 8))
# fig.add_subplot(1, 3, 1)
# plt.imshow(mark_boundaries(img, segments))
# fig.add_subplot(1, 3, 2)
# plt.imshow(sp_img)
# fig.add_subplot(1, 3, 3)
# plt.imshow(img2)
# plt.show()
# # h w c

# output = model(torch.Tensor(img2).permute(2,0,1).unsqueeze(0))
# print(torch.argmax(output, dim=1))
# probs = torch.nn.functional.softmax(output[0], dim=0)
# print(probs[162])


In [ ]:
from torchvision import datasets

dataset = datasets.ImageFolder('../datasets/tiny_imagenet_train_sub/images', transform=preprocess)

In [ ]:
print(anchor.feature_mask)
print(anchor.precision)

print(segments)
print(anchor.feature_mask)
idxs = np.argwhere(~np.isin(segments,anchor.feature_mask))
print(idxs)
# input[:, idxs[:, 0], idxs[:, 1]] = 0
# plt.imshow(input.squeeze().permute(2,1,0))

img = mark_boundaries(input.squeeze().permute(1,2,0), segments)
img[idxs[:, 0], idxs[:, 1], :] = 0
plt.imshow(img)

In [ ]:
# #h, w, c
# img.shape
print(input.shape)
model.to(device)
input[:,:, idxs[:, 0], idxs[:, 1]] = 0.34

output = model(input.to(device))
print(torch.argmax(output, dim=1))
probs = torch.nn.functional.softmax(output[0], dim=0)
print(probs[162])


In [ ]:
"""
Beam Search Test
"""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

explainer = Anchor(Tasktype.IMAGE)

def predict(x):
    test = x.to(device)
    return model(test)

anchor, segments = explainer.explain_instance(input.squeeze().permute(1,2,0), predict, "beam", 1000)